# LLM Optimization Modelling Experiment

In [3]:
import vertexai
from vertexai.preview.generative_models import GenerativeModel, Image
from IPython.display import Markdown

## 1. Define the problem description

In [136]:
problem = '''The PRODA, S.A. industrial products firm has to face the problem of scheduling
the weekly production of its three products (P1, P2 and P3). These products are
sold to large industrial firms and PRODA, S.A. wishes to supply its products in
quantities that are more profitable for it.

Each product entails three operations contributing to the costs: smelting; mechanisation; assembly and
packaging. The smelting operations for products P1 and P2 could be subcontracted, but the smelting operation for product P3 requires special equipment, thus
preventing the use of subcontracts. PRODA also want to know, how much they should subcontract.

For product P1 the direct unit costs of all possible operations are:
- smelting at PRODA: 0.30$
- subcontracted smelting: 0.50$
- mechanisation: 0.20$
- Assembly and packaging: 0.3$
The unit sales price is 1.50$.

For product P2 the direct unit costs of all possible operations are:
- smelting at PRODA: 0.50$
- subcontracted smelting: 0.60$
- mechanisation: 0.10$
- Assembly and packaging: 0.20$
The unit sales price is 1.80$.

For product P3 the direct unit costs of all possible operations are:
- smelting at PRODA: 0.40$
- mechanisation: 0.27$
- Assembly and packaging: 0.20$
The unit sales price is 1.97$.

Each unit of product P1 requires 6 min of smelting time (if performed at PRODA, S.A.), 6 min of mechanisation time and 3 min of assembly and packaging time, respectively. For product P2, the times are 10, 3 and 2 min, respectively. One unit of product P3 needs 8 min of smelting time, 8 min of mechanisation and 2 min for assembly and packaging. PRODA, S.A. has weekly capacities of 8,000 min of smelting time, 12,000 min of mechanisation time and 10,000 min of assembly and packaging time.
The objective is to maximize weekly profits.'''

## 2. Generate the mathematical model

In [144]:
#Initializing the session. To replicate, make sure the right credentials are saved in a PATH variable
PROJECT_ID = "llm4optproblems"
REGION = "us-central1"
vertexai.init(project=PROJECT_ID, location=REGION)

#Specifying the model
generative_multimodal_model = GenerativeModel("gemini-1.5-pro-preview-0409")

#The propmt applied to all problems
prompt = '''Please formulate a mathematical optimization model for this problem. Include parameters, decision variables, the objective function and the constraints in your answer.
'''

#Generate the response
response = generative_multimodal_model.generate_content([prompt+problem])


In [145]:
#Show the resopnse in a formatted way
Markdown(response.text)

## Mathematical Optimization Model for PRODA, S.A. Production Scheduling

**Parameters:**

* **Products:** 
    * P1, P2, P3
* **Operations:**
    * Smelting (Internal/Subcontracted)
    * Mechanization
    * Assembly and Packaging
* **Cost Parameters:**
    * `Cs_i` : Smelting cost of product `i` at PRODA ($/unit)
    * `Css_i`: Subcontracted smelting cost of product `i` ($/unit)
    * `Cm_i`: Mechanization cost of product `i` ($/unit)
    * `Cap_i`: Assembly and packaging cost of product `i` ($/unit)
    * `Sp_i`: Selling price of product `i` ($/unit)
* **Time Parameters:**
    * `Ts_i`: Smelting time of product `i` at PRODA (min/unit)
    * `Tm_i`: Mechanization time of product `i` (min/unit)
    * `Tap_i`: Assembly and packaging time of product `i` (min/unit)
* **Capacity Parameters:**
    * `CapS`: Total smelting capacity at PRODA (min/week)
    * `CapM`: Total mechanization capacity (min/week)
    * `CapAP`: Total assembly and packaging capacity (min/week)

**Decision Variables:**

* `Xi`: Number of units of product `i` produced per week
* `Xsi`: Number of units of product `i` smelted at PRODA per week
* `Xssi`: Number of units of product `i` smelted via subcontracting per week

**Objective Function (Maximize Weekly Profit):**

```
Maximize Z = Σ(i=1 to 3) [(Sp_i - Cs_i - Cm_i - Cap_i) * Xsi + (Sp_i - Css_i - Cm_i - Cap_i) * Xssi]
```

**Constraints:**

1. **Production Quantity:**
    * `Xi = Xsi + Xssi` for i = 1, 2, 3 (Total production equals internal + subcontracted smelting)
2. **Smelting Capacity:**
    * `Σ(i=1 to 3) Ts_i * Xsi <= CapS` (Total smelting time at PRODA cannot exceed capacity)
3. **Mechanization Capacity:**
    * `Σ(i=1 to 3) Tm_i * Xi <= CapM` (Total mechanization time cannot exceed capacity)
4. **Assembly and Packaging Capacity:**
    * `Σ(i=1 to 3) Tap_i * Xi <= CapAP` (Total assembly and packaging time cannot exceed capacity)
5. **Subcontracting Restriction:**
    * `Xssi = 0` for i = 3 (Product P3 cannot be subcontracted)
6. **Non-Negativity:**
    * `Xi, Xsi, Xssi >= 0` for i = 1, 2, 3 (Production quantities cannot be negative)

**Parameter Values (based on problem data):**

* `Cs_1 = 0.30`, `Css_1 = 0.50`, `Cm_1 = 0.20`, `Cap_1 = 0.30`, `Sp_1 = 1.50`, `Ts_1 = 6`, `Tm_1 = 6`, `Tap_1 = 3`
* `Cs_2 = 0.50`, `Css_2 = 0.60`, `Cm_2 = 0.10`, `Cap_2 = 0.20`, `Sp_2 = 1.80`, `Ts_2 = 10`, `Tm_2 = 3`, `Tap_2 = 2`
* `Cs_3 = 0.40`, `Cm_3 = 0.27`, `Cap_3 = 0.20`, `Sp_3 = 1.97`, `Ts_3 = 8`, `Tm_3 = 8`, `Tap_3 = 2`
* `CapS = 8000`, `CapM = 12000`, `CapAP = 10000`

This model can be solved using linear programming solvers to determine the optimal production quantities for each product, the amount of subcontracting, and the maximum achievable weekly profit. 


## 3. Generate the pyomo code

In [146]:
#Second prompt gets the output of the previous step and generates the code
prompt2 = "Please write pyomo code for this mathematical problem. Use sample data where needed. Indicate where you use sample data. \n"
prompt2 += response.text 
response2 = generative_multimodal_model.generate_content([prompt2])

In [147]:
#Showing the code in a formatted way
Markdown(response2.text)

```python
import pyomo.environ as pyo

# Define the model
model = pyo.ConcreteModel()

# Define sets
model.PRODUCTS = pyo.Set(initialize=['P1', 'P2', 'P3'])

# Define parameters (Sample data provided)
model.Cs = pyo.Param(model.PRODUCTS, initialize={'P1': 0.30, 'P2': 0.50, 'P3': 0.40})
model.Css = pyo.Param(model.PRODUCTS, initialize={'P1': 0.50, 'P2': 0.60, 'P3': 0})  # Sample data: Css_3 = 0 since P3 cannot be subcontracted
model.Cm = pyo.Param(model.PRODUCTS, initialize={'P1': 0.20, 'P2': 0.10, 'P3': 0.27})
model.Cap = pyo.Param(model.PRODUCTS, initialize={'P1': 0.30, 'P2': 0.20, 'P3': 0.20})
model.Sp = pyo.Param(model.PRODUCTS, initialize={'P1': 1.50, 'P2': 1.80, 'P3': 1.97})
model.Ts = pyo.Param(model.PRODUCTS, initialize={'P1': 6, 'P2': 10, 'P3': 8})
model.Tm = pyo.Param(model.PRODUCTS, initialize={'P1': 6, 'P2': 3, 'P3': 8})
model.Tap = pyo.Param(model.PRODUCTS, initialize={'P1': 3, 'P2': 2, 'P3': 2})

model.CapS = pyo.Param(initialize=8000)  # Sample data
model.CapM = pyo.Param(initialize=12000) # Sample data
model.CapAP = pyo.Param(initialize=10000) # Sample data

# Define variables
model.Xi = pyo.Var(model.PRODUCTS, within=pyo.NonNegativeReals)
model.Xsi = pyo.Var(model.PRODUCTS, within=pyo.NonNegativeReals)
model.Xssi = pyo.Var(model.PRODUCTS, within=pyo.NonNegativeReals)

# Define objective function
def objective_rule(model):
    return sum((model.Sp[i] - model.Cs[i] - model.Cm[i] - model.Cap[i]) * model.Xsi[i] + 
               (model.Sp[i] - model.Css[i] - model.Cm[i] - model.Cap[i]) * model.Xssi[i] 
               for i in model.PRODUCTS)
model.obj = pyo.Objective(rule=objective_rule, sense=pyo.maximize)

# Define constraints
model.production_qty = pyo.Constraint(model.PRODUCTS, rule=lambda model, i: model.Xi[i] == model.Xsi[i] + model.Xssi[i])
model.smelting_capacity = pyo.Constraint(rule=lambda model: sum(model.Ts[i] * model.Xsi[i] for i in model.PRODUCTS) <= model.CapS)
model.mechanization_capacity = pyo.Constraint(rule=lambda model: sum(model.Tm[i] * model.Xi[i] for i in model.PRODUCTS) <= model.CapM)
model.assembly_capacity = pyo.Constraint(rule=lambda model: sum(model.Tap[i] * model.Xi[i] for i in model.PRODUCTS) <= model.CapAP)
model.subcontracting_restriction = pyo.Constraint(rule=lambda model: model.Xssi['P3'] == 0)  # P3 cannot be subcontracted

# Solve the model
solver = pyo.SolverFactory('glpk')  # Choose a solver, e.g., glpk
results = solver.solve(model)

# Print the results
print(results)

# Display the optimal solution
print("Optimal Solution:")
for i in model.PRODUCTS:
    print(f"Product {i}:")
    print(f"  Total Production (Xi): {pyo.value(model.Xi[i])}")
    print(f"  Internal Smelting (Xsi): {pyo.value(model.Xsi[i])}")
    print(f"  Subcontracted Smelting (Xssi): {pyo.value(model.Xssi[i])}")
print(f"Maximum Weekly Profit: {pyo.value(model.obj)}") 
```

This code first defines the model, sets, parameters, variables, objective function, and constraints. Then, it calls a solver to find the optimal solution and prints the results. The sample data is incorporated into the parameter initialization. You can modify the sample data as needed.


## 4. Input problem data and try running the generated code

In [149]:
import pyomo.environ as pyo

# Define the model
model = pyo.ConcreteModel()

# Define sets
model.PRODUCTS = pyo.Set(initialize=['P1', 'P2', 'P3'])

# Define parameters (Sample data provided)
model.Cs = pyo.Param(model.PRODUCTS, initialize={'P1': 0.30, 'P2': 0.50, 'P3': 0.40})
model.Css = pyo.Param(model.PRODUCTS, initialize={'P1': 0.50, 'P2': 0.60, 'P3': 0})  # Sample data: Css_3 = 0 since P3 cannot be subcontracted
model.Cm = pyo.Param(model.PRODUCTS, initialize={'P1': 0.20, 'P2': 0.10, 'P3': 0.27})
model.Cap = pyo.Param(model.PRODUCTS, initialize={'P1': 0.30, 'P2': 0.20, 'P3': 0.20})
model.Sp = pyo.Param(model.PRODUCTS, initialize={'P1': 1.50, 'P2': 1.80, 'P3': 1.97})
model.Ts = pyo.Param(model.PRODUCTS, initialize={'P1': 6, 'P2': 10, 'P3': 8})
model.Tm = pyo.Param(model.PRODUCTS, initialize={'P1': 6, 'P2': 3, 'P3': 8})
model.Tap = pyo.Param(model.PRODUCTS, initialize={'P1': 3, 'P2': 2, 'P3': 2})

model.CapS = pyo.Param(initialize=8000)  # Sample data
model.CapM = pyo.Param(initialize=12000) # Sample data
model.CapAP = pyo.Param(initialize=10000) # Sample data

# Define variables
model.Xi = pyo.Var(model.PRODUCTS, within=pyo.NonNegativeReals)
model.Xsi = pyo.Var(model.PRODUCTS, within=pyo.NonNegativeReals)
model.Xssi = pyo.Var(model.PRODUCTS, within=pyo.NonNegativeReals)

# Define objective function
def objective_rule(model):
    return sum((model.Sp[i] - model.Cs[i] - model.Cm[i] - model.Cap[i]) * model.Xsi[i] + 
               (model.Sp[i] - model.Css[i] - model.Cm[i] - model.Cap[i]) * model.Xssi[i] 
               for i in model.PRODUCTS)
model.obj = pyo.Objective(rule=objective_rule, sense=pyo.maximize)

# Define constraints
model.production_qty = pyo.Constraint(model.PRODUCTS, rule=lambda model, i: model.Xi[i] == model.Xsi[i] + model.Xssi[i])
model.smelting_capacity = pyo.Constraint(rule=lambda model: sum(model.Ts[i] * model.Xsi[i] for i in model.PRODUCTS) <= model.CapS)
model.mechanization_capacity = pyo.Constraint(rule=lambda model: sum(model.Tm[i] * model.Xi[i] for i in model.PRODUCTS) <= model.CapM)
model.assembly_capacity = pyo.Constraint(rule=lambda model: sum(model.Tap[i] * model.Xi[i] for i in model.PRODUCTS) <= model.CapAP)
model.subcontracting_restriction = pyo.Constraint(rule=lambda model: model.Xssi['P3'] == 0)  # P3 cannot be subcontracted

# Solve the model
solver = pyo.SolverFactory('glpk')  # Choose a solver, e.g., glpk
results = solver.solve(model)

# Print the results
print(results)

# Display the optimal solution
print("Optimal Solution:")
for i in model.PRODUCTS:
    print(f"Product {i}:")
    print(f"  Total Production (Xi): {pyo.value(model.Xi[i])}")
    print(f"  Internal Smelting (Xsi): {pyo.value(model.Xsi[i])}")
    print(f"  Subcontracted Smelting (Xssi): {pyo.value(model.Xssi[i])}")
print(f"Maximum Weekly Profit: {pyo.value(model.obj)}") 


Problem: 
- Name: unknown
  Lower bound: 3680.0
  Upper bound: 3680.0
  Number of objectives: 1
  Number of constraints: 7
  Number of variables: 9
  Number of nonzeros: 19
  Sense: maximize
Solver: 
- Status: ok
  Termination condition: optimal
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: 0
      Number of created subproblems: 0
  Error rc: 0
  Time: 0.038921356201171875
Solution: 
- number of solutions: 0
  number of solutions displayed: 0

Optimal Solution:
Product P1:
  Total Production (Xi): 0.0
  Internal Smelting (Xsi): 0.0
  Subcontracted Smelting (Xssi): 0.0
Product P2:
  Total Production (Xi): 4000.0
  Internal Smelting (Xsi): 800.0
  Subcontracted Smelting (Xssi): 3200.0
Product P3:
  Total Production (Xi): 0.0
  Internal Smelting (Xsi): 0.0
  Subcontracted Smelting (Xssi): 0.0
Maximum Weekly Profit: 3680.0000000000005


## 5. Correct the code to verify model viability (optional)

## 6. Printing the outputs as strings, so they can be saved.
Those can be rendered as markdown for better readability

In [150]:
print(response.text)

## Mathematical Optimization Model for PRODA, S.A. Production Scheduling

**Parameters:**

* **Products:** 
    * P1, P2, P3
* **Operations:**
    * Smelting (Internal/Subcontracted)
    * Mechanization
    * Assembly and Packaging
* **Cost Parameters:**
    * `Cs_i` : Smelting cost of product `i` at PRODA ($/unit)
    * `Css_i`: Subcontracted smelting cost of product `i` ($/unit)
    * `Cm_i`: Mechanization cost of product `i` ($/unit)
    * `Cap_i`: Assembly and packaging cost of product `i` ($/unit)
    * `Sp_i`: Selling price of product `i` ($/unit)
* **Time Parameters:**
    * `Ts_i`: Smelting time of product `i` at PRODA (min/unit)
    * `Tm_i`: Mechanization time of product `i` (min/unit)
    * `Tap_i`: Assembly and packaging time of product `i` (min/unit)
* **Capacity Parameters:**
    * `CapS`: Total smelting capacity at PRODA (min/week)
    * `CapM`: Total mechanization capacity (min/week)
    * `CapAP`: Total assembly and packaging capacity (min/week)

**Decision Variables:*

In [151]:
print(response2.text)

```python
import pyomo.environ as pyo

# Define the model
model = pyo.ConcreteModel()

# Define sets
model.PRODUCTS = pyo.Set(initialize=['P1', 'P2', 'P3'])

# Define parameters (Sample data provided)
model.Cs = pyo.Param(model.PRODUCTS, initialize={'P1': 0.30, 'P2': 0.50, 'P3': 0.40})
model.Css = pyo.Param(model.PRODUCTS, initialize={'P1': 0.50, 'P2': 0.60, 'P3': 0})  # Sample data: Css_3 = 0 since P3 cannot be subcontracted
model.Cm = pyo.Param(model.PRODUCTS, initialize={'P1': 0.20, 'P2': 0.10, 'P3': 0.27})
model.Cap = pyo.Param(model.PRODUCTS, initialize={'P1': 0.30, 'P2': 0.20, 'P3': 0.20})
model.Sp = pyo.Param(model.PRODUCTS, initialize={'P1': 1.50, 'P2': 1.80, 'P3': 1.97})
model.Ts = pyo.Param(model.PRODUCTS, initialize={'P1': 6, 'P2': 10, 'P3': 8})
model.Tm = pyo.Param(model.PRODUCTS, initialize={'P1': 6, 'P2': 3, 'P3': 8})
model.Tap = pyo.Param(model.PRODUCTS, initialize={'P1': 3, 'P2': 2, 'P3': 2})

model.CapS = pyo.Param(initialize=8000)  # Sample data
model.CapM = pyo.Param